주식 백태스트 colab

In [ ]:
class BACKTEST:
    def __init__(self, df, symbol):
        self.example = 'example'

        #copy
        self.data = df.copy()
        self.symbol = symbol

        # figures
        self.data['Position_MACD'] = 0
        self.data['Position_RSI'] = 0
        self.data['Position_BB'] = 0
        self.data['Position_MA'] = 0
        self.data['Position_EMA'] = 0
        self.data['Position_ADX'] = 0
        self.data['Position_CCI'] = 0
        self.data['Position_Stochastic'] = 0
        self.data['Position_VWAP'] = 0
        self.data['Position_ATR'] = 0
        self.data['Position_Parabolic_SAR'] = 0
        self.data['Position_OBV'] = 0
        self.data['Position_MFI'] = 0
        self.data['Position_ROC'] = 0



    def MACD(self, short_window=2, long_window=4, signal_window=9):
      # MACD calculation
      self.data['Short_MA'] = self.data[('Close', self.symbol)].ewm(span=short_window, adjust=False).mean()
      self.data['Long_MA'] = self.data[('Close', self.symbol)].ewm(span=long_window, adjust=False).mean()
      self.data['MACD'] = self.data['Short_MA'] - self.data['Long_MA']
      self.data['Signal'] = self.data['MACD'].ewm(span=signal_window, adjust=False).mean()

      # MACD Buy/Sell signals
      self.data['MACD_prev'] = self.data['MACD'].shift(1)
      self.data['Signal_prev'] = self.data['Signal'].shift(1)
      self.data['Trade_Signal_MACD'] = 0
      self.data.loc[(self.data['MACD'] > self.data['Signal']) & (self.data['MACD_prev'] <= self.data['Signal_prev']), 'Trade_Signal_MACD'] = 1
      self.data.loc[(self.data['MACD'] < self.data['Signal']) & (self.data['MACD_prev'] >= self.data['Signal_prev']), 'Trade_Signal_MACD'] = -1

      # Fill forward the position signal for MACD strategy
      self.data['Position_MACD'] = self.data['Trade_Signal_MACD'].replace(to_replace=0, method='ffill').fillna(0)

      # Store MACD data if needed for reference
      self.MACD_data = self.data[['Short_MA', 'Long_MA', 'MACD', 'Signal', 'Trade_Signal_MACD', 'Position_MACD']].copy()

      print("MACD:",self.data['Position_MACD'].tail())

    def RSI(self,short = 0 ,window=14):
        delta = self.data[('Close', self.symbol)].diff(1)
        self.data["delta"] = delta
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(window=window, min_periods=1).mean()
        avg_loss = loss.rolling(window=window, min_periods=1).mean()
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        self.data['avg_gain'] = avg_gain
        self.data['avg_loss'] = avg_loss
        self.data['RSI'] = rsi
        self.data['Position_RSI'] = (rsi < 30).astype(int) - (rsi > 70).astype(int).replace(to_replace=0, method='ffill').fillna(0)

        if short == 0:
            self.data['Position_RSI'] = self.data['Position_RSI'].replace(-1, 0)

        print("RSI:",self.data['Position_RSI'].tail())

    def Bollinger_Bands(self,short =0,window=20, num_std=2):
        rolling_mean = self.data[('Close', self.symbol)].rolling(window=window).mean()
        rolling_std = self.data[('Close', self.symbol)].rolling(window=window).std()
        self.data['BB_upper'] = rolling_mean + (rolling_std * num_std)
        self.data['BB_lower'] = rolling_mean - (rolling_std * num_std)
        # Define Position_BB with replacement for ffill and filling NaNs with 0
        self.data['Position_BB'] = (
            ((self.data[('Close', self.symbol)] < self.data['BB_lower']).astype(int) -
             (self.data[('Close', self.symbol)] > self.data['BB_upper']).astype(int))
            .replace(to_replace=0, method='ffill').fillna(0)
        )

        if short == 0:
          # forward-fill 이후에 -1 값을 0으로 바꿔줌
          self.data['Position_BB'] = self.data['Position_BB'].replace(-1, 0)

        print("BB:" , self.data['Position_BB'].tail(8))

    def Moving_Average(self,short =0,window=50):
        self.data['MA'] = self.data[('Close', self.symbol)].rolling(window=window).mean()
        self.data['Position_MA'] = ((self.data[('Close', self.symbol)] > self.data['MA']).astype(int) -
                                    (self.data[('Close', self.symbol)] < self.data['MA']).astype(int))
        if short == 0:
            self.data['Position_MA'] = self.data['Position_MA'].replace(-1, 0)

        print("MA:",self.data['Position_MA'].tail())
    def EMA(self,short=0, window=50):
        self.data['EMA'] = self.data[('Close', self.symbol)].ewm(span=window, adjust=False).mean()
        self.data['Position_EMA'] = ((self.data[('Close', self.symbol)] > self.data['EMA']).astype(int) -
                                     (self.data[('Close', self.symbol)] < self.data['EMA']).astype(int)).replace(to_replace=0, method='ffill').fillna(0)

        if short == 0:
            self.data['Position_EMA'] = self.data['Position_EMA'].replace(-1, 0)

        print("EMA:",self.data['Position_EMA'].tail())

    def ADX(self,short=0, window=14):
        high = self.data[('High', self.symbol)]
        low = self.data[('Low', self.symbol)]
        close = self.data[('Close', self.symbol)]
        plus_dm = high.diff().clip(lower=0)
        minus_dm = -low.diff().clip(upper=0)
        tr = pd.concat([high - low, abs(high - close.shift()), abs(low - close.shift())], axis=1).max(axis=1)
        atr = tr.rolling(window=window).mean()
        plus_di = 100 * (plus_dm.ewm(alpha=1/window).mean() / atr)
        minus_di = 100 * (minus_dm.ewm(alpha=1/window).mean() / atr)
        dx = (abs(plus_di - minus_di) / (plus_di + minus_di)) * 100
        adx = dx.ewm(alpha=1/window).mean()
        self.data['ADX'] = adx
        self.data['Position_ADX'] = (adx > 25).astype(int).replace(to_replace=0, method='ffill').fillna(0)

        if short == 0:
            self.data['Position_ADX'] = self.data['Position_ADX'].replace(-1, 0)

        print("ADX:",self.data['Position_ADX'].tail())

    def CCI(self,short=0, window=20):
        tp = (self.data[('High', self.symbol)] + self.data[('Low', self.symbol)] + self.data[('Close', self.symbol)]) / 3
        sma = tp.rolling(window=window).mean()
        mad = tp.rolling(window=window).apply(lambda x: np.mean(np.abs(x - np.mean(x))), raw=True)
        cci = (tp - sma) / (0.015 * mad)
        self.data['CCI'] = cci
        self.data['Position_CCI'] = ((cci < -100).astype(int) - (cci > 100).astype(int)).replace(to_replace=0, method='ffill').fillna(0)

        if short == 0:
            self.data['Position_CCI'] = self.data['Position_CCI'].replace(-1, 0)

    def Stochastic(self, k_window=14, d_window=3):
        low_min = self.data[('Low', self.symbol)].rolling(window=k_window).min()
        high_max = self.data[('High', self.symbol)].rolling(window=k_window).max()
        self.data['%K'] = 100 * ((self.data[('Close', self.symbol)] - low_min) / (high_max - low_min))
        self.data['%D'] = self.data['%K'].rolling(window=d_window).mean()
        self.data['Position_Stochastic'] = ((self.data['%K'] > self.data['%D']).astype(int) -
                                            (self.data['%K'] < self.data['%D']).astype(int)).replace(to_replace=0, method='ffill').fillna(0)

    def VWAP(self):
        typical_price = (self.data[('High', self.symbol)] + self.data[('Low', self.symbol)] + self.data[('Close', self.symbol)]) / 3
        vwap = (typical_price * self.data[('Volume', self.symbol)]).cumsum() / self.data[('Volume', self.symbol)].cumsum()
        self.data['VWAP'] = vwap
        self.data['Position_VWAP'] = ((self.data[('Close', self.symbol)] > vwap).astype(int) -
                                      (self.data[('Close', self.symbol)] < vwap).astype(int))



    def ATR(self, window=14):
        high = self.data[('High', self.symbol)]
        low = self.data[('Low', self.symbol)]
        close = self.data[('Close', self.symbol)]
        tr = pd.concat([high - low, abs(high - close.shift()), abs(low - close.shift())], axis=1).max(axis=1)
        atr = tr.rolling(window=window).mean()
        self.data['ATR'] = atr
        self.data['Position_ATR'] = ((close > atr).astype(int) - (close < atr).astype(int)).replace(to_replace=0, method='ffill').fillna(0)


        print("ATR:",self.data['Position_ATR'].tail())

    def Parabolic_SAR(self, step=0.02, max_step=0.2):
        psar = self.data[('Close', self.symbol)].copy()
        psar_direction = 1
        psar_af = step
        extreme_point = self.data[('High', self.symbol)].iloc[0]
        for i in range(1, len(self.data)):
            psar[i] = psar[i - 1] + psar_af * (extreme_point - psar[i - 1])
            if psar_direction == 1:
                if self.data[('Low', self.symbol)].iloc[i] < psar[i]:
                    psar_direction = -1
                    psar[i] = extreme_point
                    psar_af = step
                    extreme_point = self.data[('Low', self.symbol)].iloc[i]
                else:
                    extreme_point = max(extreme_point, self.data[('High', self.symbol)].iloc[i])
                    psar_af = min(psar_af + step, max_step)
            else:
                if self.data[('High', self.symbol)].iloc[i] > psar[i]:
                    psar_direction = 1
                    psar[i] = extreme_point
                    psar_af = step
                    extreme_point = self.data[('High', self.symbol)].iloc[i]
                else:
                    extreme_point = min(extreme_point, self.data[('Low', self.symbol)].iloc[i])
                    psar_af = min(psar_af + step, max_step)
        self.data['Position_Parabolic_SAR'] = ((self.data[('Close', self.symbol)] > psar).astype(int) -
                                               (self.data[('Close', self.symbol)] < psar).astype(int)).replace(to_replace=0, method='ffill').fillna(0)

    def OBV(self):
        obv = (np.sign(self.data[('Close', self.symbol)].diff()) * self.data[('Volume', self.symbol)]).fillna(0).cumsum()
        self.data['Position_OBV'] = ((obv > 0).astype(int) - (obv < 0).astype(int)).replace(to_replace=0, method='ffill').fillna(0)
        self.data['OBV'] = obv

    def MFI(self, window=14):
        typical_price = (self.data[('High', self.symbol)] + self.data[('Low', self.symbol)] + self.data[('Close', self.symbol)]) / 3
        raw_money_flow = typical_price * self.data[('Volume', self.symbol)]
        positive_flow = raw_money_flow.where(typical_price > typical_price.shift(), 0).rolling(window=window).sum()
        negative_flow = raw_money_flow.where(typical_price < typical_price.shift(), 0).rolling(window=window).sum()
        mfi = 100 - (100 / (1 + positive_flow / negative_flow))
        print("MFI :",mfi.tail())
        self.data['MFI'] = mfi
        self.data['Position_MFI'] = ((mfi < 20).astype(int) - (mfi > 80).astype(int)).replace(to_replace=0, method='ffill').fillna(0)

        # forward-fill 이후에 -1 값을 0으로 바꿔줌
        self.data['Position_MFI'] = self.data['Position_MFI'].replace(-1, 0)

    def ROC(self, window=12):
        self.data['ROC'] = ((self.data[('Close', self.symbol)] - self.data[('Close', self.symbol)].shift(window)) /
                            self.data[('Close', self.symbol)].shift(window)) * 100
        self.data['Position_ROC'] = ((self.data['ROC'] > 0).astype(int) - (self.data['ROC'] < 0).astype(int)).replace(to_replace=0, method='ffill').fillna(0)



    def calculate_returns(self, position_column):
        """
        Calculates the market return, strategy return, and cumulative returns based on a given position column.
        :param position_column: str, the column name of the trading strategy position to be evaluated.
        """
        self.data['Market_Return'] = self.data[('Close', self.symbol)].pct_change()
        self.data['Strategy_Return'] = self.data['Market_Return'] * self.data[position_column].shift(1)

        # Cumulative returns calculation
        self.data['Cumulative_Market_Return'] = (1 + self.data['Market_Return']).cumprod()
        self.data['Cumulative_Strategy_Return'] = (1 + self.data['Strategy_Return']).cumprod()

        # Final returns
        total_market_return = self.data['Cumulative_Market_Return'].iloc[-1] - 1
        total_strategy_return = self.data['Cumulative_Strategy_Return'].iloc[-1] - 1

        self.data[f'Cumulative_{position_column}_Return'] = self.data['Cumulative_Strategy_Return']

        return {
            "Total_Market_Return": total_market_return,
            "Total_Strategy_Return": total_strategy_return
        }


    def Multi_Strategy_Combined(self):
        """
        Combines multiple indicators to create a new backtest strategy.
        """
        # Calculate all strategies
        self.MACD()
        self.RSI()
        self.Bollinger_Bands()
        self.Moving_Average()
        self.EMA()
        self.ADX()
        self.CCI()
        self.Stochastic()
        self.VWAP()
        self.ATR()
        self.Parabolic_SAR()
        self.OBV()
        self.MFI()
        self.ROC()

        # Strategy 1: Combined MACD, RSI, and Bollinger Bands
        self.data['Combined_Strategy_1_Long'] = (
            (self.data['Position_MACD'] == 1) &
            (self.data['Position_RSI'] == 1) &
            (self.data['Position_BB'] == 1)
        ).astype(int)

        self.data['Combined_Strategy_1_Short'] = (
            (self.data['Position_MACD'] == -1) &
            (self.data['Position_RSI'] == -1) &
            (self.data['Position_BB'] == -1)
        ).astype(int)

        # Strategy 2: Combined Moving Average, EMA, and ADX
        self.data['Combined_Strategy_2_Long'] = (
            (self.data['Position_MA'] == 1) &
            (self.data['Position_EMA'] == 1) &
            (self.data['Position_ADX'] == 1)
        ).astype(int)

        self.data['Combined_Strategy_2_Short'] = (
            (self.data['Position_MA'] == -1) &
            (self.data['Position_EMA'] == -1) &
            (self.data['Position_ADX'] == -1)
        ).astype(int)

        # Strategy 3: Combined CCI, Stochastic, and VWAP
        self.data['Combined_Strategy_3_Long'] = (
            (self.data['Position_CCI'] == 1) &
            (self.data['Position_Stochastic'] == 1) &
            (self.data['Position_VWAP'] == 1)
        ).astype(int)

        self.data['Combined_Strategy_3_Short'] = (
            (self.data['Position_CCI'] == -1) &
            (self.data['Position_Stochastic'] == -1) &
            (self.data['Position_VWAP'] == -1)
        ).astype(int)

        # Strategy 4: Combined ATR and Parabolic SAR
        self.data['Combined_Strategy_4_Long'] = (
            (self.data['Position_ATR'] == 1) &
            (self.data['Position_Parabolic_SAR'] == 1)
        ).astype(int)

        self.data['Combined_Strategy_4_Short'] = (
            (self.data['Position_ATR'] == -1) &
            (self.data['Position_Parabolic_SAR'] == -1)
        ).astype(int)

        # Strategy 5: Combined OBV, MFI, and ROC
        self.data['Combined_Strategy_5_Long'] = (
            (self.data['Position_OBV'] == 1) &
            (self.data['Position_MFI'] == 1) &
            (self.data['Position_ROC'] == 1)
        ).astype(int)

        self.data['Combined_Strategy_5_Short'] = (
            (self.data['Position_OBV'] == -1) &
            (self.data['Position_MFI'] == -1) &
            (self.data['Position_ROC'] == -1)
        ).astype(int)

        # Strategy 6: Combined MACD, ADX, and CCI
        self.data['Combined_Strategy_6_Long'] = (
            (self.data['Position_MACD'] == 1) &
            (self.data['Position_ADX'] == 1) &
            (self.data['Position_CCI'] == 1)
        ).astype(int)

        self.data['Combined_Strategy_6_Short'] = (
            (self.data['Position_MACD'] == -1) &
            (self.data['Position_ADX'] == -1) &
            (self.data['Position_CCI'] == -1)
        ).astype(int)

        # Strategy 7: Combined RSI, VWAP, and Parabolic SAR
        self.data['Combined_Strategy_7_Long'] = (
            (self.data['Position_RSI'] == 1) &
            (self.data['Position_VWAP'] == 1) &
            (self.data['Position_Parabolic_SAR'] == 1)
        ).astype(int)

        self.data['Combined_Strategy_7_Short'] = (
            (self.data['Position_RSI'] == -1) &
            (self.data['Position_VWAP'] == -1) &
            (self.data['Position_Parabolic_SAR'] == -1)
        ).astype(int)

        # Strategy 8: Combined EMA, ATR, and ROC
        self.data['Combined_Strategy_8_Long'] = (
            (self.data['Position_EMA'] == 1) &
            (self.data['Position_ATR'] == 1) &
            (self.data['Position_ROC'] == 1)
        ).astype(int)

        self.data['Combined_Strategy_8_Short'] = (
            (self.data['Position_EMA'] == -1) &
            (self.data['Position_ATR'] == -1) &
            (self.data['Position_ROC'] == -1)
        ).astype(int)

        # Strategy 9: Combined Stochastic, MFI, and MA
        self.data['Combined_Strategy_9_Long'] = (
            (self.data['Position_Stochastic'] == 1) &
            (self.data['Position_MFI'] == 1) &
            (self.data['Position_MA'] == 1)
        ).astype(int)

        self.data['Combined_Strategy_9_Short'] = (
            (self.data['Position_Stochastic'] == -1) &
            (self.data['Position_MFI'] == -1) &
            (self.data['Position_MA'] == -1)
        ).astype(int)

        # Strategy 10: Combined BB, ADX, and OBV
        self.data['Combined_Strategy_10_Long'] = (
            (self.data['Position_BB'] == 1) &
            (self.data['Position_ADX'] == 1) &
            (self.data['Position_OBV'] == 1)
        ).astype(int)

        self.data['Combined_Strategy_10_Short'] = (
            (self.data['Position_BB'] == -1) &
            (self.data['Position_ADX'] == -1) &
            (self.data['Position_OBV'] == -1)
        ).astype(int)

        # Determine the overall position for each combined strategy
        for i in range(1, 11):
            self.data[f'Position_Combined_Strategy_{i}'] = 0
            self.data.loc[self.data[f'Combined_Strategy_{i}_Long'] == 1, f'Position_Combined_Strategy_{i}'] = 1
            self.data.loc[self.data[f'Combined_Strategy_{i}_Short'] == 1, f'Position_Combined_Strategy_{i}'] = -1
            self.data[f'Position_Combined_Strategy_{i}'] = self.data[f'Position_Combined_Strategy_{i}'].replace(to_replace=0, method='ffill').fillna(0)
            #self.data[f'Position_Combined_Strategy_{i}'] = self.data[f'Position_Combined_Strategy_{i}'].replace(-1, 0)

            if -1 in self.data[f'Position_Combined_Strategy_{i}']:
              print("yes")
            else:
              print("no")

    def calculate_all_strategies(self):
        """
        Calculates returns for each trading strategy and stores them in a dictionary.
        """
        position_columns = [
            "Position_MACD",
            "Position_RSI",
            "Position_BB",
            "Position_MA",
            "Position_EMA",
            "Position_ADX",
            "Position_CCI",
            "Position_Stochastic",
            "Position_VWAP",
            "Position_ATR",
            "Position_Parabolic_SAR",
            "Position_OBV",
            "Position_MFI",
            "Position_ROC"
        ]

        # Add combined strategies
        for i in range(1, 11):
            position_columns.append(f'Position_Combined_Strategy_{i}')

        results = {}

        for strategy in position_columns:
            results[strategy] = self.calculate_returns(strategy)

        return results




    def calculate_all_strategies_2(self):
        """
        Calculates returns for each trading strategy and stores them in a dictionary.
        """
        position_columns = [
                                "Position_MACD",
                                "Position_RSI",
                                "Position_BB",
                                "Position_MA",
                                "Position_EMA",
                                "Position_ADX",
                                "Position_CCI",
                                "Position_Stochastic",
                                "Position_VWAP",
                                "Position_ATR",
                                "Position_Parabolic_SAR",
                                "Position_OBV",
                                "Position_MFI",
                                "Position_ROC"
                            ]

        #strategies = [col for col in self.data.columns if 'Position_' in col]
        results = {}

        for strategy in position_columns:
            print(strategy,"\tstrategy:", self.calculate_returns(strategy))
            results[strategy] = self.calculate_returns(strategy)
        #print(self.data.columns)
        return results


    def deep_learning(self):
        import pandas as pd
        import numpy as np
        import tensorflow as tf
        from sklearn.model_selection import train_test_split
        from sklearn.preprocessing import StandardScaler
        from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

        # 타겟 변수 생성: 수익률이 0 이상인 경우 1, 그렇지 않으면 0
        self.data['y'] = (self.data['Market_Return'] >= 0).astype(int)

        # NaN과 무한 값 제거
        self.data = self.data.replace([np.inf, -np.inf], np.nan).dropna()

        # 필요한 feature와 target 설정
        #features = ['Short_MA', 'Long_MA', 'Signal', 'RSI', 'BB_upper', 'BB_lower',
        #            'MA', 'EMA', 'ADX', 'CCI', 'VWAP', 'ATR', 'OBV', 'MFI', 'ROC']
        features = ["Short_MA", "Long_MA", "MFI"]
        X = self.data[features]
        y = self.data['y']

        # 데이터 전처리
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # LSTM에 맞게 시계열 데이터 형태로 변경 (예: 과거 10개 데이터로 다음 값 예측)
        time_steps = 10
        X_sequences = []
        y_sequences = []

        for i in range(time_steps, len(X_scaled)):
            X_sequences.append(X_scaled[i-time_steps:i])
            y_sequences.append(y.iloc[i])

        X_sequences, y_sequences = np.array(X_sequences), np.array(y_sequences)

        # 훈련/테스트 데이터 분할
        X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

        # 모델 생성 (LSTM 기반)
        model = tf.keras.Sequential([
            tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.LSTM(32),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

        # 모델 컴파일
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        # 학습률 조정 및 조기 종료 콜백
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)

        # 모델 학습
        model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test),
                  callbacks=[early_stopping, reduce_lr])

        # 모델 평가
        loss, accuracy = model.evaluate(X_test, y_test)
        print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")



    def deep_learning_one(self):
        self.data['y'] = (self.data['Market_Return'] >= 0 ).astype(int)

        import pandas as pd
        import numpy as np
        import tensorflow as tf
        from sklearn.model_selection import train_test_split
        from sklearn.preprocessing import StandardScaler

        # NaN과 무한 값을 제거 또는 대체
        self.data = self.data.replace([np.inf, -np.inf], np.nan).dropna()

        # 필요한 feature와 target 설정
        features = ['MFI',"Short_MA", "Long_MA"]
        X = self.data[features]

        # 목표값으로 수익률을 사용한다고 가정 (수익률 컬럼이 있는 경우)
        y = self.data['y']

        # 데이터 전처리
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # 훈련/테스트 데이터 분할
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
        print(y_train.shape,y_train)

        # 모델 생성
        model = tf.keras.Sequential([
                    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
                    tf.keras.layers.Dense(16, activation='relu'),
                    tf.keras.layers.Dropout(0.5),
                    tf.keras.layers.Dense(4, activation='relu'),
                    tf.keras.layers.Dropout(0.5),
                    tf.keras.layers.Dense(1, activation='sigmoid')
                ])

        # 모델 컴파일
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        # 모델 학습
        model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

        # 모델 평가
        loss, accuracy = model.evaluate(X_test, y_test)
        print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

    def deep_learning_xgboost(self):
        import xgboost as xgb
        from xgboost import XGBClassifier
        from sklearn.model_selection import train_test_split
        from sklearn.metrics import accuracy_score

        self.data['y'] = (self.data['Market_Return'] >= 0 ).astype(int)

        # NaN과 무한 값을 제거 또는 대체
        self.data = self.data.replace([np.inf, -np.inf], np.nan).dropna()

        # 필요한 feature와 target 설정
        features = ['Short_MA', 'Long_MA', 'Signal', 'RSI', 'BB_upper', 'BB_lower',
                    'MA', 'EMA', 'ADX', 'CCI', 'VWAP', 'ATR', 'OBV', 'MFI', 'ROC']
        X = self.data[features]


        # 데이터 전처리 후 분할
        X = self.data[features]
        y = self.data['y']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # XGBoost 모델
        model = XGBClassifier()
        model.fit(X_train, y_train)

        # 평가
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Test Accuracy: {accuracy}")




In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import pandas as pd


class StockApp:
    def __init__(self):
        # Available stocks
        self.stocks = ['AAPL', 'META', 'AMZN', 'GOOGL', 'MSFT', 'TSLA', 'NFLX', 'NVDA', 'AMD', 'INTC', 'BITX', "SOXL"]

        # Dropdown to select stock
        self.stock_dropdown = widgets.Dropdown(
            options=self.stocks,
            description='Select Stock:',
            style={'description_width': 'initial'}
        )

        # Button to get the stock data
        self.get_data_button = widgets.Button(description="Get Stock Data")
        self.get_data_button.on_click(self.get_stock_data)

        # Button to run backtest
        self.backtest_button = widgets.Button(description="Run Backtest")
        self.backtest_button.on_click(self.run_backtest)

        # Dropdown to select strategies to plot
        self.strategy_dropdown = widgets.SelectMultiple(
            options=[],
            description='Select Strategies:',
            style={'description_width': 'initial'},
            disabled=False
        )

        # Button to plot selected strategies
        self.plot_strategies_button = widgets.Button(description="Plot Strategies")
        self.plot_strategies_button.on_click(self.plot_selected_strategies)

        # Output widget to display plot and results
        self.output = widgets.Output()

        # Display widgets
        display(self.stock_dropdown, self.get_data_button, self.backtest_button, self.strategy_dropdown, self.plot_strategies_button, self.output)

        # Placeholder for stock data and BACKTEST instance
        self.stock_data = None
        self.backtest_instance = None

    def get_stock_data(self, b):
        stock_symbol = self.stock_dropdown.value
        if stock_symbol:
            # Download the stock data for the selected symbol
            self.stock_data = yf.download(stock_symbol, period='2y', interval="1h")
            self.plot_stock_data(self.stock_data, stock_symbol)

            # Initialize the BACKTEST instance with the downloaded data
            self.backtest_instance = BACKTEST(self.stock_data, stock_symbol)

    def plot_stock_data(self, stock_data, stock_symbol):
        with self.output:
            self.output.clear_output()
            plt.figure(figsize=(10, 5))
            plt.plot(stock_data.index, stock_data['Close'], label=f'{stock_symbol} Closing Price')
            plt.title(f'{stock_symbol} Stock Price (6 mc)')
            plt.xlabel('Date')
            plt.ylabel('Price (USD)')
            plt.legend()
            plt.grid()
            plt.show()

    def run_backtest(self, b):
        if self.backtest_instance:
            # Run desired backtesting strategies
            self.backtest_instance.MACD()
            self.backtest_instance.RSI()
            self.backtest_instance.Bollinger_Bands()
            self.backtest_instance.Moving_Average()
            self.backtest_instance.EMA()
            self.backtest_instance.ADX()
            self.backtest_instance.CCI()
            self.backtest_instance.Stochastic()
            self.backtest_instance.VWAP()
            self.backtest_instance.ATR()
            self.backtest_instance.Parabolic_SAR()
            self.backtest_instance.OBV()
            self.backtest_instance.MFI()
            self.backtest_instance.ROC()
            self.backtest_instance.Multi_Strategy_Combined()

            # Get the strategy results
            results = self.backtest_instance.calculate_all_strategies()

            self.backtest_instance.deep_learning()

            # Update strategy dropdown options
            self.strategy_dropdown.options = list(results.keys())

    def plot_selected_strategies(self, b):
        selected_strategies = self.strategy_dropdown.value
        if self.backtest_instance and selected_strategies:
            with self.output:
                self.output.clear_output(wait=True)
                plt.figure(figsize=(20, 8))
                for strategy in selected_strategies:
                    plt.plot(self.backtest_instance.data.index, self.backtest_instance.data[f'Cumulative_{strategy}_Return'],
                             label=f'{strategy} Cumulative Return')

                plt.title('Backtest Strategy Cumulative Returns')
                plt.xlabel('Date')
                plt.ylabel('Cumulative Return')
                plt.legend()
                plt.grid()
                plt.show()


if __name__ == "__main__":
    app = StockApp()


Dropdown(description='Select Stock:', options=('AAPL', 'META', 'AMZN', 'GOOGL', 'MSFT', 'TSLA', 'NFLX', 'NVDA'…

Button(description='Get Stock Data', style=ButtonStyle())

Button(description='Run Backtest', style=ButtonStyle())

SelectMultiple(description='Select Strategies:', options=(), style=DescriptionStyle(description_width='initial…

Button(description='Plot Strategies', style=ButtonStyle())

...

<ipython-input-2-5ae525d2009f>:188: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_Parabolic_SAR'] = ((self.data[('Close', self.symbol)] > psar).astype(int) -
<ipython-input-2-5ae525d2009f>:193: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_OBV'] = ((obv > 0).astype(int) - (obv < 0).astype(int)).replace(to_replace=0, method='ffill').fillna(0)
<ipython-input-2-5ae525d2009f>:204: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_MFI'] = ((mfi < 20).astype(int) - (mfi > 80).astype(int)).replace(to_replace=0, method='ffill').fillna(0)
<ipython-input-2-5ae525d2009f>:212: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_ROC'] = ((self.data['ROC'] > 0).astype(int) - 

MFI : Datetime
2024-11-05 16:30:00+00:00    48.709253
2024-11-05 17:30:00+00:00    48.584946
2024-11-05 18:30:00+00:00    52.995999
2024-11-05 19:30:00+00:00    51.977038
2024-11-05 20:30:00+00:00    51.055271
dtype: float64
no
no
no
no
no
no
no
no
no
no
Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.4831 - loss: 0.7011 - val_accuracy: 0.5022 - val_loss: 0.6959 - learning_rate: 0.0010
Epoch 2/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.5269 - loss: 0.6936 - val_accuracy: 0.5022 - val_loss: 0.6956 - learning_rate: 0.0010
Epoch 3/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.5233 - loss: 0.6914 - val_accuracy: 0.4803 - val_loss: 0.6953 - learning_rate: 0.0010
Epoch 4/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.4915 - loss: 0.6968 - val_accuracy: 0.4913 - val_loss: 0.6951 - learning_rate: 0.0010
Epoch 5/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5106 - loss: 0.6951 - val_accuracy: 0.4847 - val_loss: 0.6964 - learning_rate: 0.0010
Epoch 6/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5234 - loss: 0.6947 - val_accuracy: 0.4803 - val_loss: 0.6959 - learning_rate: 0.0010
Epoch 7/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.4840 - loss: 0.6964 - val_accuracy

[*********************100%***********************]  1 of 1 completed


In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import pandas as pd


class StockApp:
    def __init__(self):
        # Available stocks
        self.stocks = ['AAPL', 'META', 'AMZN', 'GOOGL', 'MSFT', 'TSLA', 'NFLX', 'NVDA', 'AMD', 'INTC', 'BITX', "SOXL"]

        # Dropdown to select stock
        self.stock_dropdown = widgets.Dropdown(
            options=self.stocks,
            description='Select Stock:',
            style={'description_width': 'initial'}
        )

        # Button to get the stock data
        self.get_data_button = widgets.Button(description="Get Stock Data")
        self.get_data_button.on_click(self.get_stock_data)

        # Button to run backtest
        self.backtest_button = widgets.Button(description="Run Backtest")
        self.backtest_button.on_click(self.run_backtest)

        # Output widget to display plot and results
        self.output = widgets.Output()

        # Display widgets
        display(self.stock_dropdown, self.get_data_button, self.backtest_button, self.output)

        # Placeholder for stock data and BACKTEST instance
        self.stock_data = None
        self.backtest_instance = None

    def get_stock_data(self, b):
        stock_symbol = self.stock_dropdown.value
        if stock_symbol:
            # Download the stock data for the selected symbol
            self.stock_data = yf.download(stock_symbol, period='6mo', interval="1h")
            self.plot_stock_data(self.stock_data, stock_symbol)

            # Initialize the BACKTEST instance with the downloaded data
            self.backtest_instance = BACKTEST(self.stock_data, stock_symbol)

    def plot_stock_data(self, stock_data, stock_symbol):
        with self.output:
            self.output.clear_output()
            plt.figure(figsize=(10, 5))
            plt.plot(stock_data.index, stock_data['Close'], label=f'{stock_symbol} Closing Price')
            plt.title(f'{stock_symbol} Stock Price (6 mc)')
            plt.xlabel('Date')
            plt.ylabel('Price (USD)')
            plt.legend()
            plt.grid()
            plt.show()

    def run_backtest(self, b):
        if self.backtest_instance:
            # Run desired backtesting strategies
            self.backtest_instance.MACD()
            self.backtest_instance.RSI()
            self.backtest_instance.Bollinger_Bands()
            self.backtest_instance.Moving_Average()
            self.backtest_instance.EMA()
            self.backtest_instance.ADX()
            self.backtest_instance.CCI()
            self.backtest_instance.Stochastic()
            self.backtest_instance.VWAP()
            self.backtest_instance.ATR()
            self.backtest_instance.Parabolic_SAR()
            self.backtest_instance.OBV()
            self.backtest_instance.MFI()
            self.backtest_instance.ROC()
            self.backtest_instance.Multi_Strategy_Combined()

            # Get the strategy results
            results = self.backtest_instance.calculate_all_strategies()

            self.backtest_instance.deep_learning_xgboost()

            # Plot the cumulative returns of each strategy
            self.plot_backtest_results(results)

    def plot_backtest_results(self, results):
        with self.output:
            self.output.clear_output(wait=True)
            plt.figure(figsize=(12, 8))
            for strategy, result in results.items():
                plt.plot(self.backtest_instance.data.index, self.backtest_instance.data[f'Cumulative_{strategy}_Return'],
                         label=f'{strategy} Cumulative Return')

            plt.title('Backtest Strategy Cumulative Returns')
            plt.xlabel('Date')
            plt.ylabel('Cumulative Return')
            plt.legend()
            plt.grid()
            plt.show()

if __name__ == "__main__":
    app = StockApp()


Dropdown(description='Select Stock:', options=('AAPL', 'META', 'AMZN', 'GOOGL', 'MSFT', 'TSLA', 'NFLX', 'NVDA'…

Button(description='Get Stock Data', style=ButtonStyle())

Button(description='Run Backtest', style=ButtonStyle())

Output()

[*********************100%***********************]  1 of 1 completed
<ipython-input-35-97bed9640747>:42: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_MACD'] = self.data['Trade_Signal_MACD'].replace(to_replace=0, method='ffill').fillna(0)
<ipython-input-35-97bed9640747>:60: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_RSI'] = (rsi < 30).astype(int) - (rsi > 70).astype(int).replace(to_replace=0, method='ffill').fillna(0)
<ipython-input-35-97bed9640747>:74: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  ((self.data[('Close', self.symbol)] < self.data['BB_lower']).astype(int) -
<ipython-input-35-97bed9640747>:95: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_EMA'] = ((self.d

MACD: Datetime
2024-11-01 15:30:00+00:00   -1
2024-11-01 16:30:00+00:00   -1
2024-11-01 17:30:00+00:00   -1
2024-11-01 18:30:00+00:00   -1
2024-11-01 19:30:00+00:00   -1
Name: Position_MACD, dtype: int64
RSI: Datetime
2024-11-01 15:30:00+00:00    0
2024-11-01 16:30:00+00:00    0
2024-11-01 17:30:00+00:00    0
2024-11-01 18:30:00+00:00    0
2024-11-01 19:30:00+00:00    0
Name: Position_RSI, dtype: int64
BB: Datetime
2024-10-31 19:30:00+00:00    0
2024-11-01 13:30:00+00:00    0
2024-11-01 14:30:00+00:00    0
2024-11-01 15:30:00+00:00    0
2024-11-01 16:30:00+00:00    0
2024-11-01 17:30:00+00:00    0
2024-11-01 18:30:00+00:00    0
2024-11-01 19:30:00+00:00    0
Name: Position_BB, dtype: int64
MA: Datetime
2024-11-01 15:30:00+00:00    1
2024-11-01 16:30:00+00:00    0
2024-11-01 17:30:00+00:00    0
2024-11-01 18:30:00+00:00    0
2024-11-01 19:30:00+00:00    0
Name: Position_MA, dtype: int64
EMA: Datetime
2024-11-01 15:30:00+00:00    0
2024-11-01 16:30:00+00:00    0
2024-11-01 17:30:00+00:00

<ipython-input-35-97bed9640747>:188: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_Parabolic_SAR'] = ((self.data[('Close', self.symbol)] > psar).astype(int) -
<ipython-input-35-97bed9640747>:193: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_OBV'] = ((obv > 0).astype(int) - (obv < 0).astype(int)).replace(to_replace=0, method='ffill').fillna(0)
<ipython-input-35-97bed9640747>:203: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_MFI'] = ((mfi < 20).astype(int) - (mfi > 80).astype(int)).replace(to_replace=0, method='ffill').fillna(0)
<ipython-input-35-97bed9640747>:211: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_ROC'] = ((self.data['ROC'] > 0).astype(int

MACD: Datetime
2024-11-01 15:30:00+00:00   -1
2024-11-01 16:30:00+00:00   -1
2024-11-01 17:30:00+00:00   -1
2024-11-01 18:30:00+00:00   -1
2024-11-01 19:30:00+00:00   -1
Name: Position_MACD, dtype: int64
RSI: Datetime
2024-11-01 15:30:00+00:00    0
2024-11-01 16:30:00+00:00    0
2024-11-01 17:30:00+00:00    0
2024-11-01 18:30:00+00:00    0
2024-11-01 19:30:00+00:00    0
Name: Position_RSI, dtype: int64
BB: Datetime
2024-10-31 19:30:00+00:00    0
2024-11-01 13:30:00+00:00    0
2024-11-01 14:30:00+00:00    0
2024-11-01 15:30:00+00:00    0
2024-11-01 16:30:00+00:00    0
2024-11-01 17:30:00+00:00    0
2024-11-01 18:30:00+00:00    0
2024-11-01 19:30:00+00:00    0
Name: Position_BB, dtype: int64
MA: Datetime
2024-11-01 15:30:00+00:00    1
2024-11-01 16:30:00+00:00    0
2024-11-01 17:30:00+00:00    0
2024-11-01 18:30:00+00:00    0
2024-11-01 19:30:00+00:00    0
Name: Position_MA, dtype: int64
EMA: Datetime
2024-11-01 15:30:00+00:00    0
2024-11-01 16:30:00+00:00    0
2024-11-01 17:30:00+00:00

<ipython-input-35-97bed9640747>:188: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_Parabolic_SAR'] = ((self.data[('Close', self.symbol)] > psar).astype(int) -
<ipython-input-35-97bed9640747>:193: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_OBV'] = ((obv > 0).astype(int) - (obv < 0).astype(int)).replace(to_replace=0, method='ffill').fillna(0)
<ipython-input-35-97bed9640747>:203: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_MFI'] = ((mfi < 20).astype(int) - (mfi > 80).astype(int)).replace(to_replace=0, method='ffill').fillna(0)
<ipython-input-35-97bed9640747>:211: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_ROC'] = ((self.data['ROC'] > 0).astype(int

Test Accuracy: 0.5562130177514792


In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import pandas as pd


class StockApp:
    def __init__(self):
        # Available stocks
        self.stocks = ['AAPL', 'META', 'AMZN', 'GOOGL', 'MSFT', 'TSLA', 'NFLX', 'NVDA', 'AMD', 'INTC', 'BITX', "SOXL"]

        # Dropdown to select stock
        self.stock_dropdown = widgets.Dropdown(
            options=self.stocks,
            description='Select Stock:',
            style={'description_width': 'initial'}
        )

        # Button to get the stock data
        self.get_data_button = widgets.Button(description="Get Stock Data")
        self.get_data_button.on_click(self.get_stock_data)

        # Button to run backtest
        self.backtest_button = widgets.Button(description="Run Backtest")
        self.backtest_button.on_click(self.run_backtest)

        # Output widget to display plot and results
        self.output = widgets.Output()

        # Display widgets
        display(self.stock_dropdown, self.get_data_button, self.backtest_button, self.output)

        # Placeholder for stock data and BACKTEST instance
        self.stock_data = None
        self.backtest_instance = None

    def get_stock_data(self, b):
        stock_symbol = self.stock_dropdown.value
        if stock_symbol:
            # Download the stock data for the selected symbol
            self.stock_data = yf.download(stock_symbol, period='3mo', interval="1h")
            self.plot_stock_data(self.stock_data, stock_symbol)

            # Initialize the BACKTEST instance with the downloaded data
            self.backtest_instance = BACKTEST(self.stock_data, stock_symbol)

    def plot_stock_data(self, stock_data, stock_symbol):
        with self.output:
            self.output.clear_output()
            plt.figure(figsize=(10, 5))
            plt.plot(stock_data.index, stock_data['Close'], label=f'{stock_symbol} Closing Price')
            plt.title(f'{stock_symbol} Stock Price (6 mc)')
            plt.xlabel('Date')
            plt.ylabel('Price (USD)')
            plt.legend()
            plt.grid()
            plt.show()

    def run_backtest(self, b):
        if self.backtest_instance:
            # Run desired backtesting strategies
            self.backtest_instance.MACD()
            self.backtest_instance.RSI()
            self.backtest_instance.Bollinger_Bands()
            self.backtest_instance.Moving_Average()
            self.backtest_instance.EMA()
            self.backtest_instance.ADX()
            self.backtest_instance.CCI()
            self.backtest_instance.Stochastic()
            self.backtest_instance.VWAP()
            self.backtest_instance.ATR()
            self.backtest_instance.Parabolic_SAR()
            self.backtest_instance.OBV()
            self.backtest_instance.MFI()
            self.backtest_instance.ROC()
            self.backtest_instance.Multi_Strategy_Combined()

            # Get the strategy results
            results = self.backtest_instance.calculate_all_strategies()

            self.backtest_instance.deep_learning_xgboost()

            # Plot the cumulative returns of each strategy
            self.plot_backtest_results(results)

    def plot_backtest_results(self, results):
        with self.output:
            self.output.clear_output(wait=True)
            plt.figure(figsize=(20, 8))
            for strategy, result in results.items():
                print(strategy, result)

                plt.plot(self.backtest_instance.data.index, self.backtest_instance.data[f'Cumulative_{strategy}_Return'],
                         label=f'{strategy} Cumulative Return')

            plt.title('Backtest Strategy Cumulative Returns')
            plt.xlabel('Date')
            plt.ylabel('Cumulative Return')
            plt.legend()
            plt.grid()
            plt.show()

if __name__ == "__main__":
    app = StockApp()


Dropdown(description='Select Stock:', options=('AAPL', 'META', 'AMZN', 'GOOGL', 'MSFT', 'TSLA', 'NFLX', 'NVDA'…

Button(description='Get Stock Data', style=ButtonStyle())

Button(description='Run Backtest', style=ButtonStyle())

Output()

[*********************100%***********************]  1 of 1 completed
<ipython-input-35-97bed9640747>:42: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_MACD'] = self.data['Trade_Signal_MACD'].replace(to_replace=0, method='ffill').fillna(0)
<ipython-input-35-97bed9640747>:60: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_RSI'] = (rsi < 30).astype(int) - (rsi > 70).astype(int).replace(to_replace=0, method='ffill').fillna(0)
<ipython-input-35-97bed9640747>:74: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  ((self.data[('Close', self.symbol)] < self.data['BB_lower']).astype(int) -
<ipython-input-35-97bed9640747>:95: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_EMA'] = ((self.d

MACD: Datetime
2024-11-01 15:30:00+00:00    1
2024-11-01 16:30:00+00:00    1
2024-11-01 17:30:00+00:00    1
2024-11-01 18:30:00+00:00    1
2024-11-01 19:30:00+00:00    1
Name: Position_MACD, dtype: int64
RSI: Datetime
2024-11-01 15:30:00+00:00    0
2024-11-01 16:30:00+00:00    0
2024-11-01 17:30:00+00:00    0
2024-11-01 18:30:00+00:00    0
2024-11-01 19:30:00+00:00    0
Name: Position_RSI, dtype: int64
BB: Datetime
2024-10-31 19:30:00+00:00    1
2024-11-01 13:30:00+00:00    1
2024-11-01 14:30:00+00:00    1
2024-11-01 15:30:00+00:00    1
2024-11-01 16:30:00+00:00    1
2024-11-01 17:30:00+00:00    1
2024-11-01 18:30:00+00:00    1
2024-11-01 19:30:00+00:00    1
Name: Position_BB, dtype: int64
MA: Datetime
2024-11-01 15:30:00+00:00    1
2024-11-01 16:30:00+00:00    1
2024-11-01 17:30:00+00:00    1
2024-11-01 18:30:00+00:00    1
2024-11-01 19:30:00+00:00    1
Name: Position_MA, dtype: int64
EMA: Datetime
2024-11-01 15:30:00+00:00    1
2024-11-01 16:30:00+00:00    1
2024-11-01 17:30:00+00:00

<ipython-input-35-97bed9640747>:188: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_Parabolic_SAR'] = ((self.data[('Close', self.symbol)] > psar).astype(int) -
<ipython-input-35-97bed9640747>:193: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_OBV'] = ((obv > 0).astype(int) - (obv < 0).astype(int)).replace(to_replace=0, method='ffill').fillna(0)
<ipython-input-35-97bed9640747>:203: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_MFI'] = ((mfi < 20).astype(int) - (mfi > 80).astype(int)).replace(to_replace=0, method='ffill').fillna(0)
<ipython-input-35-97bed9640747>:211: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_ROC'] = ((self.data['ROC'] > 0).astype(int

MACD: Datetime
2024-11-01 15:30:00+00:00    1
2024-11-01 16:30:00+00:00    1
2024-11-01 17:30:00+00:00    1
2024-11-01 18:30:00+00:00    1
2024-11-01 19:30:00+00:00    1
Name: Position_MACD, dtype: int64
RSI: Datetime
2024-11-01 15:30:00+00:00    0
2024-11-01 16:30:00+00:00    0
2024-11-01 17:30:00+00:00    0
2024-11-01 18:30:00+00:00    0
2024-11-01 19:30:00+00:00    0
Name: Position_RSI, dtype: int64
BB: Datetime
2024-10-31 19:30:00+00:00    1
2024-11-01 13:30:00+00:00    1
2024-11-01 14:30:00+00:00    1
2024-11-01 15:30:00+00:00    1
2024-11-01 16:30:00+00:00    1
2024-11-01 17:30:00+00:00    1
2024-11-01 18:30:00+00:00    1
2024-11-01 19:30:00+00:00    1
Name: Position_BB, dtype: int64
MA: Datetime
2024-11-01 15:30:00+00:00    1
2024-11-01 16:30:00+00:00    1
2024-11-01 17:30:00+00:00    1
2024-11-01 18:30:00+00:00    1
2024-11-01 19:30:00+00:00    1
Name: Position_MA, dtype: int64
EMA: Datetime
2024-11-01 15:30:00+00:00    1
2024-11-01 16:30:00+00:00    1
2024-11-01 17:30:00+00:00

<ipython-input-35-97bed9640747>:188: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_Parabolic_SAR'] = ((self.data[('Close', self.symbol)] > psar).astype(int) -
<ipython-input-35-97bed9640747>:193: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_OBV'] = ((obv > 0).astype(int) - (obv < 0).astype(int)).replace(to_replace=0, method='ffill').fillna(0)
<ipython-input-35-97bed9640747>:203: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_MFI'] = ((mfi < 20).astype(int) - (mfi > 80).astype(int)).replace(to_replace=0, method='ffill').fillna(0)
<ipython-input-35-97bed9640747>:211: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  self.data['Position_ROC'] = ((self.data['ROC'] > 0).astype(int

Test Accuracy: 0.4878048780487805


## 딥러닝으로 시작하는 주식 딥러닝 조건만들기

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터 준비 (CSV 파일이나 다른 형식으로 데이터프레임으로 읽기)
data = pd.read_csv('stock_data.csv')  # 이미 self.data로 되어 있는 데이터를 사용한다고 가정

# 필요한 feature와 target 설정
features = ['Short_MA', 'Long_MA', 'Signal', 'RSI', 'BB_upper', 'BB_lower',
            'MA', 'EMA', 'ADX', 'CCI', 'VWAP', 'ATR', 'OBV', 'MFI', 'ROC']
X = data[features]

# 목표값으로 수익률을 사용한다고 가정 (수익률 컬럼이 있는 경우)
y = data['Profit']  # 'Profit' 컬럼이 수익률을 나타낸다고 가정

# 데이터 전처리
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 훈련/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # 수익률을 예측하기 위해 출력 노드를 하나로 설정
])

# 모델 컴파일 및 학습 설정
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 모델 학습
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)

# 테스트 데이터 평가
loss, mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {mae}')

# 예측
predictions = model.predict(X_test)

# 예측 결과를 통해 가장 높은 수익률을 찾아 해당 전략 확인
max_profit_index = np.argmax(predictions)
best_strategy = X_test[max_profit_index]
print("Best strategy features:", scaler.inverse_transform([best_strategy]))


In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import pandas as pd

class StockApp:
    def __init__(self):
        # Available stocks
        self.stocks = ['AAPL', 'META', 'AMZN', 'GOOGL', 'MSFT', 'TSLA', 'NFLX', 'NVDA', 'AMD', 'INTC']

        # Dropdown to select stock
        self.stock_dropdown = widgets.Dropdown(
            options=self.stocks,
            description='Select Stock:',
            style={'description_width': 'initial'}
        )

        # Button to get the stock data
        self.get_data_button = widgets.Button(description="Get Stock Data")
        self.get_data_button.on_click(self.get_stock_data)

        # Output widget to display plot
        self.output = widgets.Output()

        # Display widgets
        display(self.stock_dropdown, self.get_data_button, self.output)

    def get_stock_data(self, b):
        stock_symbol = self.stock_dropdown.value
        if stock_symbol:
            stock_data = yf.download(stock_symbol, period='1y')
            self.plot_stock_data(stock_data, stock_symbol)

    def plot_stock_data(self, stock_data, stock_symbol):
        with self.output:
            self.output.clear_output()
            plt.figure(figsize=(10, 5))
            plt.plot(stock_data.index, stock_data['Close'], label=f'{stock_symbol} Closing Price')
            plt.title(f'{stock_symbol} Stock Price (1 Year)')
            plt.xlabel('Date')
            plt.ylabel('Price (USD)')
            plt.legend()
            plt.grid()
            plt.show()

if __name__ == "__main__":
    app = StockApp()

Dropdown(description='Select Stock:', options=('AAPL', 'META', 'AMZN', 'GOOGL', 'MSFT', 'TSLA', 'NFLX', 'NVDA'…

Button(description='Get Stock Data', style=ButtonStyle())

Output()

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# 연산자 선택 위젯 생성
def create_operator_widget():
    return widgets.Dropdown(
        options=['+', '-', '/', '*'],
        value='+',
        description='Operator:',
    )

# 비교 연산자 선택 위젯 생성
def create_comparator_widget():
    return widgets.Dropdown(
        options=['<', '>', '='],
        value='<',
        description='Comparator:',
    )

# 조건 입력 행 생성 함수
def create_condition_row():
    options = ['MACD', 'RSI', 'BB', 'MA', 'EMA', 'ADX', 'CCI', 'Stochastic', 'VWAP', 'ATR', 'Parabolic_SAR', 'OBV', 'MFI', 'ROC']

    left_value = widgets.Dropdown(options=options, description='Left Value:')
    right_value = widgets.Dropdown(options=options, description='Right Value:')
    value_inputs = {
        'MACD': [widgets.FloatText(description='Short MA:'), widgets.FloatText(description='Long MA:'), widgets.FloatText(description='Signal MA:')],
        'RSI': [widgets.FloatText(description='Period:')],
        'BB': [widgets.FloatText(description='Period:'), widgets.FloatText(description='Standard Deviation:')],
        'MA': [widgets.FloatText(description='Period:')],
        'EMA': [widgets.FloatText(description='Period:')],
        'ADX': [widgets.FloatText(description='Period:')],
        'CCI': [widgets.FloatText(description='Period:'), widgets.FloatText(description='Constant:')],
        'Stochastic': [widgets.FloatText(description='K Period:'), widgets.FloatText(description='D Period:')],
        'VWAP': [],
        'ATR': [widgets.FloatText(description='Period:')],
        'Parabolic_SAR': [widgets.FloatText(description='Step:'), widgets.FloatText(description='Maximum:')],
        'OBV': [],
        'MFI': [widgets.FloatText(description='Period:')],
        'ROC': [widgets.FloatText(description='Period:')]
    }

    comparator = create_comparator_widget()
    delete_button = widgets.Button(description='Delete', button_style='danger')

    # 드롭다운에 따라 각기 다른 입력 필드를 표시
    def update_left_value_input(change):
        left_parameter_inputs = value_inputs.get(change['new'], [])
        left_parameter_box.children = left_parameter_inputs

    def update_right_value_input(change):
        right_parameter_inputs = value_inputs.get(change['new'], [])
        right_parameter_box.children = right_parameter_inputs

    left_parameter_box = widgets.VBox(value_inputs.get(left_value.value, []))
    right_parameter_box = widgets.VBox(value_inputs.get(right_value.value, []))

    left_value.observe(update_left_value_input, names='value')
    right_value.observe(update_right_value_input, names='value')

    condition_box = widgets.HBox([left_value, left_parameter_box, comparator, right_value, right_parameter_box, delete_button])

    def delete_condition(b):
        condition_box.close()

    delete_button.on_click(delete_condition)
    return condition_box

# 전체 위젯 구성
def create_conditions_widget():
    conditions = []
    conditions_box = widgets.VBox()
    add_button = widgets.Button(description='Add Condition', button_style='success')

    def add_condition(b):
        condition_row = create_condition_row()
        conditions.append(condition_row)
        conditions_box.children = list(conditions)

    add_button.on_click(add_condition)

    main_box = widgets.VBox([add_button, conditions_box])
    return main_box

# 위젯 표시
conditions_widget = create_conditions_widget()
display(conditions_widget)
